# Random Forest

In [ ]:
from src.trainning          import create_variables
from src.data_loader        import create_df
from src.evaluation         import show_confusion_matrix, show_metricas, show_feature_importances, show_auc_roc, show_especificidad, get_feature_importances
from src.trainning          import final_pipeline, apply_grid_search
from src.data_loader        import serialize_models

df_model_rf     = create_df('raw_data')
model_pipeline  = final_pipeline(random_forest=True)

In [ ]:
X_train, X_test, y_train, y_test = create_variables(df_model_rf)

model           = apply_grid_search(model_pipeline)

model.fit(X_train, y_train)

rf_optimizado   = model.best_estimator_
rf_model        = rf_optimizado.named_steps['model']

pred            = rf_optimizado.predict(X_test)

In [ ]:
metricas, accuracy_final    = show_metricas(y_test, pred)
especificidad               = show_especificidad(y_test, pred)

serialize_models(rf_model, 'rf_01_model')

print(f"\nAccuracy del Modelo Final Optimizado: {accuracy_final:.4f}")
print(f"\nEspecificidad del Modelo Final Optimizado: {especificidad:.4f}")
print("\nReporte de Clasificación del Modelo Final:")
print(metricas)


Accuracy del Modelo Final Optimizado: 0.8415

Especificidad del Modelo Final Optimizado: 0.9054

Reporte de Clasificación del Modelo Final:
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       222
           1       0.51      0.51      0.51        43

    accuracy                           0.84       265
   macro avg       0.71      0.71      0.71       265
weighted avg       0.84      0.84      0.84       265



**Accuracy**

A primera vista, un 0.84 es un valor muy bueno, pero en este contexto es engañoso. El dataset tiene un desbalanceo de clases muy grande. En los datos de test son 222 de la clase 0 y 43 de la clase 1. Es decir, si el modelo dijese, sin necesidad de entrenamiento, que 'Nadie se va de la empresa', ya estaria acertando un gran porcentaje de los casos, y acercandose asi al 82% de Accuracy. Por tanto, para dar por bueno esta valor, es necesario compararlo con las demás métricas.

**Recall**

Aqui el problema esta en la clase 1, los trabajadores que abandonan. El modelo es capaz de detectar unicamente el 51% de los empleados que se van. Más de la mitad de los abandonos pasan desapercibidos para el modelo.

**Precision**

Igual, la clase 0 no hay problema, pero la clase 1 es muy mala. De todos los trabajadores que el modelo marca como positivos, solo el 51% lo son.

**F1-Score**

Un 51% de F1-Score indica que el modelo no esta aprendiendo los patrones de la clase 1, pero con un 91%, si esta aprendiendo los patrones de la clase 0. Hay un equilibrio de recall y precision muy mediocre para la clase 1.

**Especificidad**

Acierta en un 90% los casos de VN(Verdaderos Negativos), es decir, que tiene un alto acierto en la gente que no se va de la empresa.

In [4]:
gr = show_confusion_matrix(y_test, pred, 'Random Forest')
gr

Verdaderos Negativos (TN): 199 (El modelo predijo correctamente que el empleado "Se queda").

Verdaderos Positivos (TP): 20 (El modelo predijo correctamente que el empleado "Se va").

Falsos Negativos (FN): 23 (El modelo dijo que el empleado "Se queda", pero en realidad se fue).

Falsos Positivos (FP): 23 (El modelo dijo que el empleado "Se va", pero en realidad se quedó).

En la matriz de confusion se observa claramente como el modelo acierta casi al completo los casos de trabajadores que no abandonan, pero falla estrepitosamente con los trabajadores que si lo hacen. Se esta pasando por alto a mas de la mitad de las bajas.

In [ ]:
gr, auc_roc = show_auc_roc(model, X_test, y_test)
gr

El modelo ha conseguido un AUC de 0.74, un resultado aceptable. Esto se traduce en que hay un 75% de posibilidades de que, escogiendo aleatoriamente a un trabajador de la clase 0 y a otro de la clase 1, el modelo le asigne una mayor probabilidad de salida al segundo. Al principio, la curva tiene una subida muy pronunciada. Esto quiere decir que, las primeras bajas las detecta perfectamente. A partir de 0.2 en la tasa de falsos positivos, es donde ya empieza a tener más problemas, y clasifica verdaderos positivos cuando realmente no lo son, por eso la curva pasa a ir horizontalmente hacia la derecha.

In [6]:
feature_names = rf_optimizado.named_steps['preprocessing'].get_feature_names_out()

fi = get_feature_importances(rf_model, feature_names)
gr = show_feature_importances(fi)
gr

# Random Forest - 15 Variables

In [ ]:
model_pipeline                      = final_pipeline(random_forest=True, f_importance=True)
X_train, X_test, y_train, y_test    = create_variables(df_model_rf)

model_fi    = apply_grid_search(model_pipeline)

model_fi.fit(X_train, y_train)

rf_optimizado   = model_fi.best_estimator_
rf_model        = rf_optimizado.named_steps['model']

pred = rf_optimizado.predict(X_test)

serialize_models(rf_model, 'rf_02_model_15v')

metricas, accuracy_final    = show_metricas(y_test, pred)
especificidad               = show_especificidad(y_test, pred)

print(f"\nAccuracy del Modelo Final Optimizado: {accuracy_final:.4f}")
print(f"\nEspecificidad del Modelo Final Optimizado: {especificidad:.4f}")
print("\nReporte de Clasificación del Modelo Final:")
print(metricas)


Accuracy del Modelo Final Optimizado: 0.8226

Especificidad del Modelo Final Optimizado: 0.8964

Reporte de Clasificación del Modelo Final:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       222
           1       0.45      0.44      0.45        43

    accuracy                           0.82       265
   macro avg       0.67      0.67      0.67       265
weighted avg       0.82      0.82      0.82       265



In [8]:
gr = show_confusion_matrix(y_test, pred, 'Random Forest')
gr

In [ ]:
gr, auc_roc = show_auc_roc(model_fi, X_test, y_test)
gr

In [10]:
feature_names = rf_optimizado.named_steps['preprocessing'].get_feature_names_out()

fi = get_feature_importances(rf_model, feature_names[:15])
gr = show_feature_importances(fi)
gr

# Random Forest - 15 Variables - Umbral Modificado

In [ ]:
y_probs = model_fi.predict_proba(X_test)[:, 1]

pred_optimo = ( y_probs >= 0.35 ).astype(int)

In [ ]:
metricas, accuracy_final = show_metricas(y_test, pred_optimo)
especificidad = show_especificidad(y_test, pred_optimo)

print(f"\nAccuracy del Modelo Final Optimizado: {accuracy_final:.4f}")
print(f"\nEspecificidad del Modelo Final Optimizado: {especificidad:.4f}")
print("\nReporte de Clasificación del Modelo Final:")
print(metricas)


Accuracy del Modelo Final Optimizado: 0.6453

Especificidad del Modelo Final Optimizado: 0.6306

Reporte de Clasificación del Modelo Final:
              precision    recall  f1-score   support

           0       0.92      0.63      0.75       222
           1       0.27      0.72      0.40        43

    accuracy                           0.65       265
   macro avg       0.60      0.68      0.57       265
weighted avg       0.82      0.65      0.69       265



In [13]:
gr = show_confusion_matrix(y_test, pred_optimo, 'R.Forest - 15 Variables - Umbral Modificado')
gr